In [ ]:
# 1. Neighbor Joining
def find_min_in_matrix(matrix, indexList):
    min_val = float('inf')
    min_index = [0,0]
    for i in range(len(indexList)):
        for j in range(len(indexList)):
            if matrix[i][j] < min_val and i != j:
                min_val = matrix[i][j]
                min_index = [i,j]
    cluster_index = [indexList[min_index[0]], indexList[min_index[1]]]
    # since we don't use numpy, here min_index indicating matrix index of the min value, cluster_index are the corresponding cluster index of the matrix index.
    return min_val, min_index, cluster_index

def modify_matrix(matrix, i, j, new_row, new_col):
    index = sorted([i, j]) # Ensure we remove the larger index first
    # Remove specified row i,j
    matrix.pop(index[1])
    matrix.pop(index[0])
    # Remove specified column i,j from remaining rows
    for row in matrix:
        row.pop(index[1])
        row.pop(index[0])
    # Add new row
    matrix.append(new_row)
    # Add new column
    for row_index, row in enumerate(matrix):
        row.append(new_col[row_index])
    return matrix

def NeighborJoining(distance_matrix: List[List[int]]) -> Dict[tuple, int]:
    n = len(distance_matrix)
    #Returns a tuple representing nodes connected with weight at value Dict[tuple]. 
    clusters = {m:[m] for m in range(n)}
    # T is the collection of weighted edges connecting each node
    T = {}
    current = n
    # special case if there is only 2 cols
    if n == 2:
        return {(1,0):distance_matrix[1][0]}
    # if there is more than 2 cols
    while len(clusters) > 2:
        # build the NJ-matrix of input distance matrix
        totalDistance = [sum(distance_matrix[i]) for i in range(len(distance_matrix))]
        D_star = [[0 for _ in range(len(distance_matrix))] for _ in range(len(distance_matrix))]
        for i in range(len(distance_matrix)):
            for j in range(len(distance_matrix)):
                if i != j:
                    D_star[i][j] = (len(distance_matrix)-2)*(distance_matrix[i][j])-totalDistance[i]-totalDistance[j]
        # find the smallest element and its corresponding index in matrix and cluster index
        _ , matrix_index, cluster_index = find_min_in_matrix(D_star, list(clusters.keys()))
        i = matrix_index[0]
        j = matrix_index[1]
        delta = (totalDistance[i]-totalDistance[j])/(len(distance_matrix)-2)
        limb_i = (distance_matrix[i][j]+delta)/2
        limb_j = (distance_matrix[i][j]-delta)/2
        # add the edge and the corresponding limb length
        T[(current, cluster_index[0])] = limb_i
        T[(current, cluster_index[1])] = limb_j
        
        restCol = [k for k in range(len(distance_matrix)) if k not in matrix_index]
        new_row = [(distance_matrix[i][k] + distance_matrix[j][k]-distance_matrix[i][j]) / 2 for k in restCol]
        new_col = new_row.copy()
        new_col.append(0)
        # remove the rows and columns of the closest two clusters in the distance matrix 
        # add new row and columns representing the new cluster into the distance matrix
        distance_matrix = modify_matrix(distance_matrix, i, j, new_row, new_col)
        # remove the two clusters from the clusters list and store the information into the new cluster
        # add the new cluster into the cluster relationship record
        new_cluster = clusters.pop(cluster_index[0])+ clusters.pop(cluster_index[1])
        clusters[current] = new_cluster
        current += 1
    rest = list(clusters.keys())
    T[(rest[1], rest[0])] = distance_matrix[0][1]
        
    return T


# 2. 
def SmallParsimony(leaves_labels: List[str],
                               edge_list: List[Tuple[int, int]]) -> Tuple[int, List[str]]:

    alphabet = 'ACGT'
    leaves_no = len(leaves_labels) 
    node_set = set(i[j] for i in edge_list for j in range(2))
    root = max(node_set)
    
    scores = {node: {base: [float("inf")] * len(leaves_labels[0]) for base in alphabet} for node in range(2 * leaves_no - 1)}
    # traverse every nodes except for leaves
    backtrack = {node: [''] * len(leaves_labels[0]) for node in range(leaves_no, 2 * leaves_no - 1)} 
    
   # childList store the sons and daughters of any node except for leaves
    childList = {}
    for edge in edge_list:
        if edge[0] not in childList:
            childList[edge[0]] = []
        childList[edge[0]].append(edge[1])
    
    def cal_score(node, base, position):
        if node < leaves_no:  # Leaf node
            return 0 if leaves_labels[node][position] == base else float('inf')
        if scores[node][base][position] != float('inf'):  # Already calculated
            return scores[node][base][position]
        score = 0
        for child in childList[node]:
            child_score = min(cal_score(child, next_base, position) + (next_base != base) for next_base in alphabet)
            score += child_score
        scores[node][base][position] = score
        return score
    
    min_score = 0
    # do small parsimony for every character of the string
    for i in range(len(leaves_labels[0])):
        # for every internal node
        for j in childList:
            for base in alphabet:
                scores[j][base][i] = cal_score(j, base, i)
        
        root_base, root_score = min(scores[root].items(), key=lambda x: x[1][i])
        min_score += root_score[i]
        backtrack[root][i] = root_base

    labels = [''] * (2 * leaves_no - 1)
    def label_tree(node, position):
        # nodes except for leaves needs label
        if node >= leaves_no:  
            base = backtrack[node][position]
            labels[node] += base
            for child in childList[node]:
                child_base = min(alphabet, key=lambda b: scores[child][b][position] + (b != base))
                if child >= leaves_no:  
                    backtrack[child][position] = child_base  # Assign the base for backtracking
                label_tree(child, position)
                
    for i in range(leaves_no):
        labels[i] = leaves_labels[i]
    for position in range(len(leaves_labels[0])):
        label_tree(root, position)
        
    result_edges = [labels[parent] for parent in childList.keys()]

    return min_score, labels



# 3. 
def SmallParsimonyUnrootedTree(leaves_labels: List[str],
                               edge_list: List[Tuple[int, int]]) -> Tuple[int, List[str]]:

    alphabet = 'ACGT'
    leaves_no = len(leaves_labels) 
    node_set = set(i[j] for i in edge_list for j in range(2))
    largest_node = max(node_set)
    node_set.remove(largest_node)
    second_largest_node = max(node_set)
    # generate a root to record all scores
    root = largest_node + 1
    
    scores = {node: {base: [float("inf")] * len(leaves_labels[0]) for base in alphabet} for node in range(2 * leaves_no - 1)}
    # traverse every nodes except for leaves
    backtrack = {node: [''] * len(leaves_labels[0]) for node in range(leaves_no, 2 * leaves_no - 1)} 
    edge_list.remove((second_largest_node,largest_node))
    edge_list.extend([(second_largest_node, root),(largest_node, root)])

   # childList store the sons and daughters of any node except for leaves
    childList = {}
    for edge in edge_list:
        if edge[1] not in childList:
            childList[edge[1]] = []
        childList[edge[1]].append(edge[0])
    
    def cal_score(node, base, position):
        if node < leaves_no:  # Leaf node
            return 0 if leaves_labels[node][position] == base else float('inf')
        if scores[node][base][position] != float('inf'):  # Already calculated
            return scores[node][base][position]
        score = 0
        for child in childList[node]:
            child_score = min(cal_score(child, next_base, position) + (next_base != base) for next_base in alphabet)
            score += child_score
        scores[node][base][position] = score
        return score
    
    min_score = 0
    # do small parsimony for every character of the string
    for i in range(len(leaves_labels[0])):
        # for every internal node
        for j in childList:
            for base in alphabet:
                scores[j][base][i] = cal_score(j, base, i)
        
        root_base, root_score = min(scores[root].items(), key=lambda x: x[1][i])
        min_score += root_score[i]
        backtrack[root][i] = root_base

    labels = [''] * (2 * leaves_no - 1)
    def label_tree(node, position):
        # nodes except for leaves needs label
        if node >=leaves_no:  
            base = backtrack[node][position]
            labels[node] += base
            for child in childList[node]:
                child_base = min(alphabet, key=lambda b: scores[child][b][position] + (b != base))
                if child >= leaves_no:  
                    backtrack[child][position] = child_base  # Assign the base for backtracking
                label_tree(child, position)
                
    for i in range(leaves_no):
        labels[i] = leaves_labels[i]
    for position in range(len(leaves_labels[0])):
        label_tree(root, position)
        
    result_edges = [labels[parent] for parent in childList.keys()]

    return min_score, labels[:root]